In [11]:
#!pip install tweepy==3.10.0

  Attempting uninstall: tweepy
    Found existing installation: tweepy 4.4.0
    Uninstalling tweepy-4.4.0:
      Successfully uninstalled tweepy-4.4.0


In [1]:
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import socket
import json

In [2]:
# Academic Twitter account Authentication
consumer_key = 'LZC1g2KxGOtoAqEP7tdw0D46k'
consumer_secret = "rcihpYubdrrb8P2wZisL00xD2o4USlWmqWI8rultxBykQ5GJGj"
bearerToken = "AAAAAAAAAAAAAAAAAAAAACzVXwEAAAAAweqVt0121BJcg9hOJWkYI83Vy8w%3DsIioo0WoyjDuVpd8niHogXHhHnTeRSpiUujn7AvxS41RqwQntu"
access_token = "1060338061085736960-2vwceLQvsIKbFpCleT2eswqHZGOnDT"
access_secret = "zZmAWYHAcrh70L8kpibtoGlOrPuJJQpAtFyXrrLR0sweC"

In [3]:
class TweetsListener(StreamListener):
    # Class to listen the tweets

    def __init__(self, csocket):
        self.client_socket = csocket
    
    def on_data(self, data):
        try:
            msg = json.loads(data)
            print("new message")
            
            # if tweet is longer than 140 characters
            if "extended_tweet" in msg:
                # add at the end of each tweet "t_end" 
                self.client_socket\
                    .send(str(msg['extended_tweet']['full_text']+"t_end")\
                    .encode('utf-8'))
                print(msg['extended_tweet']['full_text'])
                
            else:
                # add at the end of each tweet "t_end" 
                self.client_socket\
                    .send(str(msg['text']+"t_end")\
                    .encode('utf-8'))
                print(msg['text'])
                
            return True
        
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    
    def on_error(self, status):
        print(status)
        return True

In [4]:
def sendData(c_socket, keyword):
    print('start sending data from Twitter to socket')
    
    # authentication based on the credentials
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    
    # start sending data from the Streaming API 
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    twitter_stream.filter(locations=[-78.6,33.95,-69.14,43.47], track = keyword, languages=["en"])

In [5]:
import os

netcat = 'xfce-terminal --command "nc -nvlp 5555"'
print ("Starting listener on port 5555")
from subprocess import call
call(netcat,shell=True)

Starting listener on port 5555


1

In [6]:
if __name__ == "__main__":
    # server (local machine) creates listening socket
    s = socket.socket()
    host = "127.0.0.1"    
    port = 5555
    s.bind((host, port))
    print('socket is ready')
    
    # server (local machine) listens for connections
    s.listen(4)
    print('socket is listening')
    
    # return the socket and the address on the other side of the connection (client side)
    c_socket, addr = s.accept()
    print("Received request from: " + str(addr))
    
    # the keyword for the tweet data is selected here
    sendData(c_socket, keyword = ['vaccination'])

socket is ready
socket is listening
Received request from: ('127.0.0.1', 59342)
start sending data from Twitter to socket
new message
@jhen_1 Out for the season. Had surgery on a fractured ankle yesterday. 😢
https://t.co/mQTaiUaOvU
new message
RT @KDVR: Natural immunity was six times stronger during the delta wave than vaccination, according to a new report from the U.S. Centers f…
new message
Me watching this: "Wow what a run by Derrick Hen... wait that's not him"
new message
RT @Poppyjuice: Watching the marches against mandatory vaccination for NHS staff and I'm angry.

A long and rambling thread follows:
new message
RT @Poppyjuice: So, watching healthcare staff who have refused to take this vaccine, when they knew they would be around vulnerable patient…
new message
RT @PeterDawsonUSA: #Israel study shows Covid19 infection rate for vaccinated people is 138.4 times that of those w #NaturalImmunity having…
new message
@YowieMark @usnjkpolk @newscomauHQ Where's the evidence? Some count

Facts!!!!
new message
Looming Scandal: HOW MANY NFL PLAYERS ARE PLAYING WITH FAKE VACCINATION CARDS? 
Mina Kimes says the NFL should model Antonio Brown's punishment for using a fake vaccine card to Evander Kane's suspension in the NHL if found guilty 
https://t.co/WhRVjIgahD 08
new message
RT @kksheld: Vaccine A was made to stop Virus A.
Virus A mutated into Virus B.
Vaccine A does not stop Virus B.
The executive branch of the…
new message
@SenSanders I assume you feel the same about one’s choice regarding vaccination?
new message
Should Geneva trust Ember? #artvideos #art https://t.co/cEPc1pWjnP
new message
RT @BryanDeanWright: Biden is banning unvaxxed Mexican &amp; Canadian truckers from delivering goods to American stores. 

Starting today.
new message
@AllOneProblem @TitusNation Well, if I am vaccinated I don’t care about others. Because that is what vaccination does, keep one safe.
new message
Anyone not taking the science, data, &amp; realities of vaccination to help with the P

new message
RT @blackintheempir: I could have sworn they told us this was misinformation
new message
RT @mtracey: The US government, in conjunction with other national governments, continues to use vaccination status as a condition for free…
new message
RT @kksheld: Vaccine A was made to stop Virus A.
Virus A mutated into Virus B.
Vaccine A does not stop Virus B.
The executive branch of the…
new message
Let's go!
new message
RT @KDVR: Natural immunity was six times stronger during the delta wave than vaccination, according to a new report from the U.S. Centers f…
new message
@D_AmandaNicole Be more the happy to be by your side!
new message
@jakethedegen That's super dope!
new message
Tannehill has money on the Bengals.. I'm convinced now. 
#Titans #TitanUp
new message
RT @ClayTravis: Twitter banned many people for sharing facts like these.
new message
We will see how this goes
new message
@TheeDougFather Thanks. Sinuses are bothering me and getting a headache, but that could be normal 

new message
@bubblegumkids_ ✌️lots of love shared of bubblegum kids on @NFT365podcast https://t.co/FqhCIj4d2p
new message
Tannehill choking on a Popeyes biscuit with no drink out there
new message
RT @ZubyMusic: Reuters article recognising the power of natural immunity (19th January 2022)
https://t.co/oFKRZIbyXW
new message
I'm at @OTTO_pizza in Andover, MA https://t.co/QsG6Ivc8Pu
new message
RT @Poppyjuice: Watching the marches against mandatory vaccination for NHS staff and I'm angry.

A long and rambling thread follows:
new message
HOW IS YOUNG SHELDON STILL A SHOW?! 

ANYBODY THAT WATCHES THIS, YOU ARE A PROBLEM.
new message
Great opportunity for those who won’t be skiing @breckenridgemtn on Friday!

#PedsICU
@JHU_NeuroPICU
new message
RT @JeromeAdamsMD: 3700 deaths, 90% unvaccinated, and yet people still are sticking with “omicron is mild,” and “I’m done with covid.” 

We…
new message
This Definitely Came on Time 💅🏾💅🏾💅🏾 Amen #HarlemEverAfter https://t.co/FttABuzaAx
new message
@CG

new message
Life is so funny 😭
new message
RT @Poppyjuice: Watching the marches against mandatory vaccination for NHS staff and I'm angry.

A long and rambling thread follows:
new message
Cue the Band 🚨

#WeWantMore | 🎥: @grayson_jons https://t.co/t2opyDVYUN
new message
@queersurgeon I’m here for this outfit 😍 a whole vibe!
new message
Free Casanova - King Renz 🌪🌪 @KawanPrather https://t.co/J5T2nkbcst
new message
Police began to separate the groups, several people tried to get passed police lines. Police had to keep them apart. 
The groups faced off with megaphones, and more police arrived. https://t.co/3P8mHxFptS
new message
Yet again, shoulda kept mike hilton , at whatever cost
new message
RT @jimmy_dore: Good to see the CDC catching up to @joerogan.
new message
@Midnightsmoke1 @RalphGarman @ThatKevinSmith @FlappersComedy Damn fucking right
new message
Ten Hot Tips for Looking Sexier While Swabbing Your Nostrils https://t.co/vXsD1zv84t via @wellesley_news
new message
Warm-ups with @H

new message
I’ve been in mourning all day. Grief is complex. My heart is heavy. My cousin has left this plane. 31 years feels far too short, but I didn’t write her story.
new message
RT @kksheld: Vaccine A was made to stop Virus A.
Virus A mutated into Virus B.
Vaccine A does not stop Virus B.
The executive branch of the…
new message
I love my #CB90 fam and thanks for your support! I’m happy @DukesMayoBowl found a great image created by Coastal Marsh Creations for me to give as a present to Coach Beamer… There’s no reason to be upset and let’s just remember that great WIN against the team from the North! 🤙🏻
new message
RT @TimKennedyMMA: Man it’s almost like this was a conspiracy theory six months ago.
new message
@snarkgrapefruit They actually called taunting in a playoff game. Come on, Blakeman.
new message
@maryderrickart @maxhart74116542 @glissy61 @RonFilipkowski @USSupremeCourt @GOP @GOPL @LeaderMcConnell @HouseGOP @SenateGOP Yet, liberals scream my body my choice pertaining to ab

new message
RT @Connybub: 👇🇸🇪👏👊👍
Vaccination card protests in Europe attract thousands of people. In Sweden, where vaccination certificates are require…
new message
Ok but people who don’t believe in vaccination and modern medicine should be barred from being health workers in general, not just in covid. I don’t want to be treated by them.
new message
“CDC is studying the impact of vaccination, boosters and prior infection during the Omicron surge and expects to issue further reports when that data becomes available.”
new message
RT @morganisawizard: quite a few people owe quite a few apologies 🤧
new message
RT @CDCgov: Parents &amp; Caregivers: Children and teens ages 5 years &amp; up should get a #COVID19 vaccine. Knowing what to expect can help make…
new message
@MollyJongFast Oh well 🤷‍♀️don’t feel sorry for anyone at this point vaccination works it’s proven
new message
@PanasonicDX4500 @ESRosebud @ClaireSassman @Twitter Everyone knows this. The RT queen 👑
new message
RT @dwallacew

new message
RT @VicGovDH: If you’re 18 and over, there’s still time to walk-up and get your third dose at state-run vaccination centres until Monday 24…
new message
RT @TimLonghurst: Hey journalists - here's a question cheat-sheet from a member of the community!

https://t.co/VAMdhnin92

#Letitrip4kids…
new message
@dandakich Greatest GM in the league Dan!
new message
@AntarcticEchoes @AmyYma7027 @_Navaros_ I love your artwork, next time I’ll order from you. Just let me know 💗
new message
Three Days Grace - I Hate Everything About You (Official Video) https://t.co/qsy2Eotwlc via @sindoneofficial
new message
Farts. #Bengals
new message
Back at Adelard where @MSCAthletics and @GoBlueClippers meet in a Division I rivalry clash. Mount is 1-4 and Cumberland is 2-4-1.
Lot of people in the barn tonight, which is great to see. https://t.co/EbpbW80VLn
new message
Can you recommend anyone for this job? Managing Director https://t.co/ovU0Lh0WNi Salisbury, MD
new message
Titans are really surprisi

new message
At CVS Health, we are committed to fostering diversity and inclusion in all aspects of our organization. Check out our available opportunities in #Newtown, CT today. https://t.co/u14kb1w99Q #pharmtech
new message
RT @richardursomd: Natural immunity deniers still exist and some of them still act like this is bombshell news. 
It’s not a bombshell. It’s…
new message
I'm at Anthony's Coal Fired Pizza - @anthonyscfpizza in Woodbury, NY https://t.co/W0F3noocr5
new message
The older I get, the more I identify with Magneto.
new message
RT @BryanDeanWright: Biden is banning unvaxxed Mexican &amp; Canadian truckers from delivering goods to American stores. 

Starting today.
new message
Treadmill broke down for the second time in a few months, but @NordicTrack @iFit #iconfitness are in no hurry to ship a new part. Who’d want to run indoors when it’s 15 degrees out anyway?
new message
Lowell leads Merrimack 1-0 after 1 period. Shots on goal are 8-7 in favor of Lowell, shot attempts are

KeyboardInterrupt: 